In [8]:
import os
import json
import ase.io
from tqdm import tqdm
from glob import glob
from pathlib import Path
import pandas as pd

In [9]:
split = "test"

In [10]:
path = f"/net/csefiles/coc-fung-cluster/Shuyi/Playground/LLM4StructGen/data/mp20-cif/{split}.csv"
df = pd.read_csv(path)

In [11]:
cif_path = f"/net/csefiles/coc-fung-cluster/Shuyi/Datasets/CDVAE/mp_20/raw_{split}/"
cifs = glob(cif_path + "*.cif")

In [12]:
out = []

for cif in tqdm(cifs):
    mp_id = Path(cif).stem
    atoms = ase.io.read(cif)
    row = df[df["material_id"] == mp_id]

    out_dict = {
        "material_id": mp_id,
        "atomic_numbers": atoms.get_atomic_numbers().tolist(),
        "atomic_symbols": atoms.get_chemical_symbols(),
        "positions": atoms.get_positions().tolist(),
        "cell": atoms.get_cell().tolist(),
        "band_gap": float(row['band_gap'].values[0]),
        "formation_energy_per_atom": float(row['formation_energy_per_atom'].values[0]),
        "e_above_hull": float(row['e_above_hull'].values[0]),
        "spacegroup_number": int(row['spacegroup.number'].values[0]),
        "pretty_formula": row['pretty_formula'].values[0],
        "elements": list(set(atoms.get_chemical_symbols())),
    }

    out.append(out_dict)

100%|██████████| 9046/9046 [00:32<00:00, 281.18it/s]


In [13]:
# save as json file
with open(f"{split}.json", "w") as f:
    json.dump(out, f)